<a href="https://colab.research.google.com/github/vmorozov/colab/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Private GitHub Setup
1. Create a personal access token (classic) with the **repo** scope at https://github.com/settings/tokens.
2. Keep the token secret—do not share it or commit it to source control.
3. When prompted below, enter your GitHub username and paste the token; the notebook stores it only for this session.
4. If you prefer, set the variables ahead of time with `%env GITHUB_USERNAME=<user>` and `%env GITHUB_TOKEN=<token>`.
5. Similarly you can `%env PROJECT_NAME=<your github project>`

In [ ]:
!git clone https://github.com/vmorozov/colab.git

In [ ]:

import getpass
import os

if "GITHUB_USERNAME" not in os.environ or not os.environ["GITHUB_USERNAME"].strip():
    os.environ["GITHUB_USERNAME"] = input("Enter your GitHub username: ").strip()

if "GITHUB_TOKEN" not in os.environ or not os.environ["GITHUB_TOKEN"].strip():
    os.environ["GITHUB_TOKEN"] = getpass.getpass("Enter your GitHub personal access token: ").strip()

if not os.environ["GITHUB_TOKEN"]:
    raise RuntimeError("GITHUB_TOKEN is required to clone the private repository.")

In [ ]:
from colab.setup.project import (
    change_directory,
    clone_or_update_repo,
    ensure_project_name,
    execute_notebook,
    install_project_requirements,
    setup_google_drive_project,
    setup_vscode_tunnel,
    store_credentials_from_env,
    run_notebook_in_background,
)

In [ ]:
netrc_path = store_credentials_from_env()
print(f"Credentials stored in {netrc_path}")

In [ ]:
#download github project
project_name = ensure_project_name()
project_dir = clone_or_update_repo("vmorozov", project_name)
print(f"Repository ready at {project_dir}")

In [ ]:
current_dir = change_directory(project_dir)
print(f"Working directory set to: {current_dir}")

index_url = install_project_requirements()
print(f"Using PyG wheel index: {index_url}")

In [ ]:
#pull the last changes from remote github into current directory from command line. overwrite local changes
#!git pull --force --all

In [ ]:
#or in google drive will be auomaticly named after project and added to environment
#project_dir = setup_google_drive_project()

In [ ]:
#set directory to save results
#either inside project code
out_dir = os.path.join(project_dir, "out")
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    os.environ['OUT_DIR'] = out_dir
print(f"Output directory set to: {out_dir}")

In [ ]:
my_notebook = 'eval.ipynb'
# output_path = execute_notebook(my_notebook, os.path.join(out_dir, my_notebook))
# print(f"Notebook execution complete. Output saved to {output_path}")
#OR convert to script and run in background
run_notebook_in_background(my_notebook, os.path.join(out_dir, my_notebook.replace('.ipynb', '.output.log')))


In [ ]:
# !git config user.email "you@example.com"
# !git config user.name "$GITHUB_USERNAME" && git add -A && git commit -m "done" && git push

In [ ]:
# vscode_tunnel = setup_vscode_tunnel()
# vscode_tunnel.connect(name=project_name, create_new_project=project_name)